In [1]:
!pip install -q "smolagents[litellm]" \
                "langchain" \
                "openai>=1.0.0" \
                "datasets" \
                "faiss-cpu" \
                "sentence-transformers" \
                "python-dotenv"

#### 목표! 내부 금융 문서를 질문/답변할 수 있는 RAG 파이프라인을 구축하고, 이를 smol-agent의 '도구'로 장착하여 웹 검색 능력까지 갖춘 고성능 리서치 Agent를 완성한다.

In [2]:
import os
from dotenv import load_dotenv

# .env 파일을 만들어 OPENAI_API_KEY="sk-..." 형식으로 키를 저장해주세요.
load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    print("⚠️ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인해주세요!")
else:
    print("✅ API 키가 성공적으로 로드되었습니다.")

✅ API 키가 성공적으로 로드되었습니다.


### 단계 1: 데이터 준비 및 'RAG 파이프라인' 구축
Agent에게 알려줄 우리만의 '지식 창고'를 만드는 과정입니다. Hugging Face의 금융 뉴스 데이터셋을 불러와 검색 가능한 벡터 DB로 만들겠습니다.

In [7]:
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. 데이터셋 로드 (사진에서 본 데이터셋)
print("💾 데이터셋을 로드합니다...")
dataset = load_dataset("nmixx-fin/synthetic_financial_report_korean", split="train")

# 튜토리얼을 위해 데이터 일부만 사용 (예: 100개)
documents = dataset.select(range(100))['text']
print(f"✅ {len(documents)}개의 금융 뉴스 리포트를 로드했습니다.")

# 2. 텍스트 분할기(Chunker) 준비
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # 각 조각의 최대 글자 수
    chunk_overlap=50, # 조각 간 겹치는 글자 수
    length_function=len,
)

# 3. 문서들을 작은 조각으로 분할
print("\n✂️ 문서를 작은 조각으로 분할합니다...")
chunks = text_splitter.create_documents(documents)
print(f"✅ 총 {len(chunks)}개의 조각으로 분할되었습니다.")

# 분할된 조각 예시 확인
print("\n--- 분할된 조각 예시 ---")
print(chunks[0].page_content)

💾 데이터셋을 로드합니다...
✅ 100개의 금융 뉴스 리포트를 로드했습니다.

✂️ 문서를 작은 조각으로 분할합니다...
✅ 총 151개의 조각으로 분할되었습니다.

--- 분할된 조각 예시 ---
KOSPI는 1.8% 하락하여 2,454포인트를 기록했습니다. 장 초반 낙폭이 줄어들었으나, 이후 다시 하락세로 돌아섰습니다. 현재 시장에는 뚜렷한 방향성이 결여되어 있습니다.

미국 주식시장은 강력한 고용 지표 발표 이후 경기 우려가 완화되었으며, S&P500은 새로운 최고치를 경신했습니다. JOLTs 구인이직 보고서에 따르면, 10월 구인건수는 774만 건으로 예상치인 751만 건을 초과하였고, 9월의 737만 건에 비해 개선되었습니다. 이러한 안정적인 경제 지표는 12월 금리 인하 확률을 72.9%로 낮추는 데 기여했습니다.

KOSPI는 정치적 상황의 영향으로 상승 종목이 105개인 반면, 하락 종목은 800개에 달했습니다. 52주 신고가는 2개에 불과하며, 신저가는 91개로 나타났습니다. 최근 대통령의 비상계엄 선포 이후 국회가 계엄 해제 결의를 하였고, 대통령이 해제 요구를 수용하는 역사적 정치적 사건이 발생했습니다.


### 문서 조각을 벡터로 변환 (OpenAI Embedding)

분할된 텍스트 조각들을 OpenAI의 text-embedding-3-small 모델을 사용해 숫자 벡터(임베딩)로 변환합니다.

In [4]:
import numpy as np
from openai import OpenAI

client = OpenAI()

print("🧠 OpenAI Embedding API를 사용해 문서 조각들을 벡터로 변환합니다...")

# page_content만 추출하여 리스트로 만듭니다.
chunk_texts = [chunk.page_content for chunk in chunks]

# OpenAI API 호출
response = client.embeddings.create(
    input=chunk_texts,
    model="text-embedding-3-small"
)

# 임베딩 결과(벡터 리스트)를 추출
embeddings = [res.embedding for res in response.data]

# NumPy 배열로 변환 (FAISS에서 사용하기 위함)
embedding_matrix = np.array(embeddings).astype("float32")

print(f"✅ {embedding_matrix.shape[0]}개의 벡터를 생성했습니다. (각 벡터의 차원: {embedding_matrix.shape[1]})")

🧠 OpenAI Embedding API를 사용해 문서 조각들을 벡터로 변환합니다...
✅ 151개의 벡터를 생성했습니다. (각 벡터의 차원: 1536)


### 벡터 검색 엔진 구축 (FAISS)

변환된 벡터들을 FAISS 인덱스에 저장하여, 특정 질문과 가장 유사한 문서 조각을 초고속으로 찾을 수 있는 검색 엔진을 만듭니다.

In [6]:
import faiss

print("🚀 FAISS로 벡터 검색 엔진(인덱스)을 구축합니다...")

# 벡터의 차원 수를 가져옵니다.
d = embedding_matrix.shape[1]

# 가장 기본적인 FAISS 인덱스인 IndexFlatL2를 생성합니다.
index = faiss.IndexFlatL2(d)

# 인덱스에 벡터들을 추가합니다.
index.add(embedding_matrix)

print(f"✅ FAISS 인덱스가 성공적으로 구축되었습니다. 총 {index.ntotal}개의 벡터가 저장되었습니다.")

🚀 FAISS로 벡터 검색 엔진(인덱스)을 구축합니다...
✅ FAISS 인덱스가 성공적으로 구축되었습니다. 총 151개의 벡터가 저장되었습니다.


### RAG 파이프라인을 '도구(Tool)'로 만들기
이제 위에서 만든 모든 과정을 하나의 함수로 묶고, @tool 데코레이터를 붙여 smol-agent가 사용할 수 있는 도구로 만듭니다.

In [8]:
from smolagents import tool

@tool
def financial_report_inspector(query: str, k: int = 3) -> str:
    """
    금융 문서에 대한 질문에 답변할 때 사용하세요.
    웹 검색으로는 절대 찾을 수 없는 고급 금융 리포트 정보입니다.
    예를 들어 '1분기 매출액은?', '최근 보고서의 핵심 내용은?' 같은 질문에 사용됩니다.
    
    Args:
        query (str): 사용자의 질문.
        k (int): 검색할 관련 문서 조각의 수. 기본값은 3입니다.
    """
    print(f"👉 RAG Tool executed: financial_report_inspector(query='{query}')")
    
    # 1. 사용자의 질문(query)을 벡터로 변환
    query_response = client.embeddings.create(input=[query], model="text-embedding-3-small")
    query_vector = np.array([query_response.data[0].embedding]).astype("float32")
    
    # 2. FAISS 인덱스에서 가장 유사한 k개의 문서 조각 검색
    distances, indices = index.search(query_vector, k)
    
    # 3. 검색된 문서 조각들의 원본 텍스트를 조합
    results = [chunks[i].page_content for i in indices[0]]
    
    # 결과를 하나의 문자열로 합쳐서 반환
    return "\n---\n".join(results)

print("✅ 'financial_report_inspector' RAG 도구가 성공적으로 생성되었습니다!")

✅ 'financial_report_inspector' RAG 도구가 성공적으로 생성되었습니다!


### 최종 Agent 구성 및 실행
open-deep-research에서 사용했던 Manager/Worker 구조에, 우리가 만든 RAG 도구와 웹 검색 도구를 모두 장착하여 최종 '하이브리드 리서치 Agent'를 만듭니다.

In [13]:
from smolagents import CodeAgent, ToolCallingAgent, LiteLLMModel, WebSearchTool

# LLM 모델 정의
model = LiteLLMModel(model_id="gpt-4.1-mini")

# 1. Worker: 웹 검색을 담당하는 Agent
#    (Serper API 키가 필요합니다. 없다면 이 Agent는 제외하고 실행하세요.)
web_agent = ToolCallingAgent(
    model=model,
    tools=[WebSearchTool()],
    name="web_search_agent",
    description="최신 정보나 외부 웹사이트 정보가 필요할 때 사용하는 웹 검색 전문가입니다.",
    verbosity_level=2,
)

# 2. Manager: RAG 도구와 Worker를 모두 관리하는 최종 Agent
manager_agent = CodeAgent(
    model=model,
    tools=[financial_report_inspector], # 우리가 만든 RAG 도구를 직접 장착!
    managed_agents=[web_agent], # 웹 검색 Agent를 관리하도록 설정
    verbosity_level=2,
)

print("✅ 최종 하이브리드 리서치 Agent가 구성되었습니다.")

✅ 최종 하이브리드 리서치 Agent가 구성되었습니다.


In [14]:
# 내부 문서 검색(RAG)과 외부 웹 검색(Web Search)이 모두 필요한 질문
complex_question = "보고서에 따르면 '미국 증시'에 대한 전망이 어떤가요? 그리고 실제로 현재 미국 주요 지수(S&P 500)는 어떻게 되고 있는지 웹에서 검색해서 비교해주세요."

print(f"🤔 복합 질문: {complex_question}")
print("\n🚀 Agent가 리서치를 시작합니다...")
print("-" * 30)

# 최종 Agent 실행
final_answer = manager_agent.run(complex_question)

🤔 복합 질문: 보고서에 따르면 '미국 증시'에 대한 전망이 어떤가요? 그리고 실제로 현재 미국 주요 지수(S&P 500)는 어떻게 되고 있는지 웹에서 검색해서 비교해주세요.

🚀 Agent가 리서치를 시작합니다...
------------------------------


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 보고서에 따르면 '미국 증시'에 대한 전망이 어떤가요? 그리고 실제로 현재 미국 주요 지수(S&P 500)는 어떻게 되고    │
│ 있는지 웹에서 검색해서 비교해주세요.                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will first use the financial_report_inspector tool to get the outlook on '미국 증시' from the financial 
report. Then I will use the web_search_agent tool to get the current status of the S&P 500 index from the web to   
compare with the report's outlook.                                                                                 
                                                                                                                   
<code>                                                                                                             
financial_outlook = financial_report_inspector(query="미국 증시 전망", k=3)                                        
print("Financial report outlook on 미국 증시:", financial_outlook)                                                 
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  financial_outlook = financial_report_inspector(query="미국 증시 전망", k=3)                                      
  print("Financial report outlook on 미국 증시:", financial_outlook)                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The financial report outlook describes a positive and optimistic sentiment on the US stock market with    
recent gains in major indices including the S&P 500. However, the most recent web search information shows that the
S&P 500 index is experiencing a slight decline of about 0.1% due to cautious investor sentiment influenced by      
tariff policy uncertainties and geopolitical factors. The report's positive outlook and the current slight dip seem
to reflect a short-term volatility within an overall optimistic longer-term market sentiment.                      
                                                                                                                   
I will now consolidate these findings into a final answer.                                                         
                                                                                                                   
<code>                                                                                                             
final_summary = (                                                                                                  
    "The financial report indicates a positive outlook for the US stock market, "                                  
    "highlighting renewed rallies, strong performance in technology and retail stocks, "                           
    "and optimism driven by economic indicators and expected interest rate cuts. "                                 
    "However, the current status of the S&P 500 as per the latest web search shows a slight decline "              
    "of about 0.1%, influenced by caution over ongoing trade tariff discussions and geopolitical uncertainties. "  
    "This suggests short-term volatility in the market amidst an overall optimistic longer-term perspective."      
[38;2;230;237;243;48;2;13;

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_summary = (                                                                                                
      "The financial report indicates a positive outlook for the US stock market, "                                
      "highlighting renewed rallies, strong performance in technology and retail stocks, "                         
      "and optimism driven by economic indicators and expected interest rate cuts. "                               
      "However, the current status of the S&P 500 as per the latest web search shows a slight decline "            
      "of about 0.1%, influenced by caution over ongoing trade tariff discussions and geopolitical uncertainties.  
  "                                                                                                                
      "This suggests short-term volatility in the market amidst an overall optimistic longer-term perspective."    
  )                                                                                                                
                                                                                                                   
  final_answer(final_summary)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The financial report indicates a positive outlook for the US stock market, highlighting renewed
rallies, strong performance in technology and retail stocks, and optimism driven by economic indicators and 
expected interest rate cuts. However, the current status of the S&P 500 as per the latest web search shows a slight
decline of about 0.1%, influenced by caution over ongoing trade tariff discussions and geopolitical uncertainties. 
This suggests short-term volatility in the market amidst an overall optimistic longer-term perspective.

[Step 3: Duration 4.38 seconds| Input tokens: 8,899 | Output tokens: 468]

In [16]:
import json

print("="*30)
print("✨ 최종 리서치 답변 ✨")
print("="*30)
print(final_answer)


✨ 최종 리서치 답변 ✨
The financial report indicates a positive outlook for the US stock market, highlighting renewed rallies, strong performance in technology and retail stocks, and optimism driven by economic indicators and expected interest rate cuts. However, the current status of the S&P 500 as per the latest web search shows a slight decline of about 0.1%, influenced by caution over ongoing trade tariff discussions and geopolitical uncertainties. This suggests short-term volatility in the market amidst an overall optimistic longer-term perspective.


In [17]:

# Agent의 전체 작업 기록을 보기 쉽게 출력
print("\n" + "="*30)
print("🧠 Agent 팀의 협업 과정 (Memory) 엿보기")
print("="*30)

agent_transcript_objects = manager_agent.write_memory_to_messages()
transcript_as_dicts = []
for msg in agent_transcript_objects:
    message_dict = {
        'role': msg.role,
        'content': msg.content if hasattr(msg, 'content') and msg.content is not None else ''
    }
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        message_dict['tool_calls'] = msg.tool_calls
    if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
        message_dict['tool_call_id'] = msg.tool_call_id
    transcript_as_dicts.append(message_dict)

print(json.dumps(transcript_as_dicts, indent=2, ensure_ascii=False))


🧠 Agent 팀의 협업 과정 (Memory) 엿보기
[
  {
    "role": "system",
    "content": [
      {
        "type": "text",
        "text": "You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', '<code>', and 'Observation:' sequences.\n\nAt each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the '<code>' sequence, you should write the code in simple Python. The code sequence must end with '</code>' sequence.\nDuring each intermediate step, you can use 'print()' to save whatever important information you will then need.\nThese print outputs will then appear in the 'Observation:' field, which will be available as in